In [7]:
import numpy as np
import torch
import pickle
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd
from pathlib import Path

# Import your Model class and utilities
from utils.match_prediction.model import Model
from utils.match_prediction import (
    MODEL_PATH,
    ENCODERS_PATH,
    MODEL_CONFIG_PATH,
    get_best_device,
)
from utils.match_prediction.champions import Champion

# Load model and move to device
device = get_best_device()

# Load label encoders
with open(ENCODERS_PATH, "rb") as f:
    label_encoders = pickle.load(f)

# Load model configuration
with open(MODEL_CONFIG_PATH, "rb") as f:
    model_params = pickle.load(f)

# Initialize model
model = Model(
    num_categories=model_params["num_categories"],
    num_champions=171,  # TODO: load from champion enum
    embed_dim=model_params["embed_dim"],
    dropout=model_params["dropout"],
    hidden_dims=model_params["hidden_dims"],
)

# Load model weights
state_dict = torch.load(MODEL_PATH, map_location=device, weights_only=True)
fixed_state_dict = {
    k.replace("_orig_mod.", ""): state_dict[k] for k in state_dict.keys()
}
model.load_state_dict(fixed_state_dict)
model.to(device)
model.eval()

# Extract champion-patch embeddings
embeddings = model.champion_patch_embedding.weight.detach().cpu().numpy()
num_patches = model.num_patches
print(f"num_patches: {num_patches}")
num_champions = model.num_champions



# Create mapping from champion ID to name
id_to_name = {champ.id: champ.display_name for champ in Champion}

# Get patch values from model
patch_values = model.patch_values.numpy()

# Prepare data for visualization
champion_patch_data = []
for champ_idx in range(num_champions):
    champ_id = label_encoders["champion_ids"].classes_[champ_idx]
    try:
        champ_id = int(champ_id)
        champ_name = id_to_name.get(champ_id, f"Unknown ID {champ_id}")
    except ValueError:
        champ_name = str(champ_id)

    for patch_idx in range(num_patches):
        embed_idx = champ_idx * num_patches + patch_idx
        patch_value = patch_values[patch_idx]
        embedding = embeddings[embed_idx]

        champion_patch_data.append(
            {
                "embedding": embedding,
                "champion": champ_name,
                "patch": f"{patch_value:.2f}",
                "label": f"{champ_name} ({patch_value:.2f})",
            }
        )

# Convert embeddings to 2D using t-SNE
all_embeddings = np.stack([data["embedding"] for data in champion_patch_data])
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(all_embeddings)

# Create DataFrame for plotting
df = pd.DataFrame(
    {
        "x": embeddings_2d[:, 0],
        "y": embeddings_2d[:, 1],
        "Champion": [data["champion"] for data in champion_patch_data],
        "Patch": [data["patch"] for data in champion_patch_data],
        "Label": [data["label"] for data in champion_patch_data],
    }
)

# Create interactive plot with Plotly
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="Champion",
    hover_data=["Champion", "Patch"],
    title="Champion-Patch Embedding Visualization (t-SNE)",
    labels={"x": "t-SNE Component 1", "y": "t-SNE Component 2"},
)

# Update traces for better visualization
fig.update_traces(marker=dict(size=8), selector=dict(mode="markers"))

# Update layout
fig.update_layout(
    showlegend=True,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.02),
    width=1200,
    height=800,
)

# Show plot
fig.show()

# Optional: Calculate and display patch-to-patch distances for each champion
print("\nAnalyzing patch-to-patch distances for champions:")
for champ_idx in range(num_champions):
    champ_id = label_encoders["champion_ids"].classes_[champ_idx]
    try:
        champ_id = int(champ_id)
        champ_name = id_to_name.get(champ_id, f"Unknown ID {champ_id}")
    except ValueError:
        champ_name = str(champ_id)

    patch_embeds = embeddings[champ_idx * num_patches : (champ_idx + 1) * num_patches]
    max_dist = np.max(np.linalg.norm(patch_embeds[:, None] - patch_embeds, axis=2))
    print(f"{champ_name}: Max distance between patches = {max_dist:.4f}")

Model dimensions:
- Categorical features: 1
- Champion positions: 10
- Numerical features projection: 1
- Total embedded features: 13
- Embedding dimension: 128
- MLP input dimension: 1664
num_patches: 23



Analyzing patch-to-patch distances for champions:
Annie: Max distance between patches = 1.1053
Kayle: Max distance between patches = 1.2566
Xerath: Max distance between patches = 1.0968
Shyvana: Max distance between patches = 1.1049
Ahri: Max distance between patches = 0.7823
Graves: Max distance between patches = 0.6433
Fizz: Max distance between patches = 1.0146
Volibear: Max distance between patches = 0.8305
Rengar: Max distance between patches = 1.0520
Master Yi: Max distance between patches = 1.1100
Varus: Max distance between patches = 0.8264
Nautilus: Max distance between patches = 0.6877
Viktor: Max distance between patches = 0.8355
Sejuani: Max distance between patches = 0.9436
Fiora: Max distance between patches = 0.8804
Ziggs: Max distance between patches = 1.0341
Lulu: Max distance between patches = 0.6195
Draven: Max distance between patches = 0.7821
Alistar: Max distance between patches = 1.0608
Hecarim: Max distance between patches = 0.9985
Kha'Zix: Max distance between

In [10]:
# Function to visualize a single champion's patch embeddings
def visualize_champion_patches(champion_name: str):
    # Filter data for the specified champion
    champion_df = df[df["Champion"] == champion_name].copy()

    if len(champion_df) == 0:
        print(f"Champion '{champion_name}' not found in the dataset")
        return

    # Create figure
    fig = px.scatter(
        champion_df,
        x="x",
        y="y",
        text="Patch",  # Show patch numbers as labels
        title=f"{champion_name} Embedding Changes Across Patches",
        labels={"x": "t-SNE Component 1", "y": "t-SNE Component 2"},
    )

    # Update scatter points
    fig.update_traces(
        marker=dict(size=12, color="blue"),  # Larger markers since we have fewer points
        textposition="top center",
        textfont=dict(size=10),
    )

    # Add lines connecting consecutive patches
    fig.add_trace(
        go.Scatter(
            x=champion_df["x"],
            y=champion_df["y"],
            mode="lines",
            line=dict(color="gray", width=1, dash="dot"),
            showlegend=False,
        )
    )

    # Update layout
    fig.update_layout(
        showlegend=False,
        width=800,
        height=600,
        title_x=0.5,  # Center the title
        hovermode="closest",
    )

    # Calculate and display the maximum distance between any two patches
    patch_embeds = embeddings[df[df["Champion"] == champion_name].index]
    max_dist = np.max(np.linalg.norm(patch_embeds[:, None] - patch_embeds, axis=2))
    print(
        f"\nMaximum distance between any two patches for {champion_name}: {max_dist:.4f}"
    )

    # Show plot
    fig.show()


# Don't forget to import go
import plotly.graph_objects as go

# Example usage - replace with any champion name from your dataset
visualize_champion_patches("Rell")

# Optional: Print available champion names
print("\nAvailable champions:")
print(sorted(df["Champion"].unique()))


Maximum distance between any two patches for Rell: 0.8449



Available champions:
['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Ambessa', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'Aurelion Sol', 'Aurora', 'Azir', 'Bard', "Bel'Veth", 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn', 'Camille', 'Cassiopeia', "Cho'Gath", 'Corki', 'Darius', 'Diana', 'Dr. Mundo', 'Draven', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Hwei', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'Jarvan IV', 'Jax', 'Jayce', 'Jhin', 'Jinx', "K'Sante", "Kai'Sa", 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', "Kha'Zix", 'Kindred', 'Kled', "Kog'Maw", 'LeBlanc', 'Lee Sin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'Master Yi', 'Mel', 'Milio', 'Miss Fortune', 'Mordekaiser', 'Morgana', 'Naafiri', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu & Wil